In [ ]:
# Mount Google Drive to access dataset
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

!ls '/content/drive/MyDrive/Colab Notebooks/Client data'

data_path = '/content/drive/MyDrive/Colab Notebooks/Client data/dataset_folder'


# Set path to the dataset
data_path = '/content/drive/MyDrive/Colab Notebooks/Client data'

# Install necessary libraries
!pip install tensorflow
!pip install scikit-learn
!pip install keras

# Import libraries
import os
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import DenseNet121
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout


images, labels = load_and_preprocess_data(data_path)

# Function to load and preprocess images
def load_and_preprocess_data(data_path, img_size=(224, 224)):
    classes = os.listdir(data_path)
    images = []
    labels = []

    for i, class_name in enumerate(classes):
        class_path = os.path.join(data_path, class_name)
        for image_name in os.listdir(class_path):
            image_path = os.path.join(class_path, image_name)
            img = cv2.imread(image_path)
            img = cv2.resize(img, img_size)
            img = img / 255.0  # Normalize pixel values
            images.append(img)
            labels.append(i)

    return np.array(images), np.array(labels)
# Load and preprocess data
images, labels = load_and_preprocess_data(data_path)

# Split the dataset into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(images, labels, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=40,  # Increase rotation range
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


# Create DenseNet121 model
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.5))  # Add Dropout layer with a dropout rate of 0.5
model.add(Dense(3, activation='softmax'))  # Assuming 3 classes: cholangiocarcinoma, HCC, Normal liver

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Reduce learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model with data augmentation
batch_size = 32
epochs = 10

# Add early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    datagen.flow(X_train, y_train, batch_size=batch_size),
    steps_per_epoch=len(X_train) // batch_size,
    epochs=epochs,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping]
)

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X_test, y_test)
train_accuracy = history.history['accuracy'][-1]
val_accuracy = history.history['val_accuracy'][-1]

print(f"Training Accuracy: {train_accuracy}")
print(f"Validation Accuracy: {val_accuracy}")
print(f"Test Accuracy: {test_accuracy}")

# Plot training history
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

model.save('/content/drive/MyDrive/Colab Notebooks/Client data')

Mounted at /content/drive
 cholangiocarcinoma   DenseNet121.h5   HCC  'Normal Liver'


NotADirectoryError: [Errno 20] Not a directory: '/content/drive/MyDrive/Colab Notebooks/Client data/DenseNet121.h5'

# DenseNet121  
# Training Accuracy : 97.06%
# Validation Accuracy : 95.24%
# Test Accuracy : 90.48% !pip install tensorflow



In [ ]:
from google.colab import drive
import os
import shutil
from sklearn.model_selection import train_test_split

# Mount Google Drive
drive.mount('/content/drive')

# Set the path to your dataset folders in Google Drive
dataset_path = '/content/drive/MyDrive/Colab Notebooks/Client data'

# Set the path where you want to save the train and test sets in Google Drive
save_path = '/content/drive/MyDrive/Train & Test'